# Tableau-Ready Dataset

This notebook combines the clean irrigation data with all derived variables created during analysis for easy use in Tableau dashboards.

In [1]:
import pandas as pd
import numpy as np

print("="*80)
print("CREATING TABLEAU-READY IRRIGATION DATASET")
print("="*80)

CREATING TABLEAU-READY IRRIGATION DATASET


In [2]:
# Load the cleaned dataset
print("n1. Loading cleaned data...")
df = pd.read_excel('Irrigation_DS_Clean.xlsx')
print(f"   ✓ Loaded {len(df)} rows with {len(df.columns)} columns")

n1. Loading cleaned data...
   ✓ Loaded 30 rows with 55 columns


## Create Derived Cost Variables

In [3]:
print("2. Creating derived cost variables...")

# Define cost columns for aggregation
cost_columns = [
    'land_prep_grand_total_cost',
    'nursery_prep_total_cost',
    'cultivation_labor_total_cost',
    'fertilizer_total_cost',
    'production_final_harvest_cost_yer',
    'pest_control_grand_total_cost',
    'water_grand_total_cost'
]

# Calculate total cost per record
df['total_cost'] = df[cost_columns].sum(axis=1)
print(f"   ✓ Created 'total_cost' variable")

# Define benefit metric (production quantity)
df['production_kg'] = df['production_quantity_baskets_kg']
print(f"   ✓ Created 'production_kg' variable")

# Calculate cost-benefit metrics
# Cost per kg (avoiding division by zero)
df['cost_per_kg'] = np.where(
    df['production_kg'] > 0,
    df['total_cost'] / df['production_kg'],
    np.nan
)
print(f"   ✓ Created 'cost_per_kg' variable")

# Production efficiency (kg per 1000 YER)
df['production_efficiency'] = np.where(
    df['total_cost'] > 0,
    df['production_kg'] / df['total_cost'] * 1000,
    np.nan
)
print(f"   ✓ Created 'production_efficiency' variable")

2. Creating derived cost variables...
   ✓ Created 'total_cost' variable
   ✓ Created 'production_kg' variable
   ✓ Created 'cost_per_kg' variable
   ✓ Created 'production_efficiency' variable


## Create Additional Derived Variables

In [4]:
print("3. Creating additional derived variables...")

# Cost category percentages
for col in cost_columns:
    pct_col_name = f'{col}_pct'
    df[pct_col_name] = np.where(
        df['total_cost'] > 0,
        (df[col] / df['total_cost']) * 100,
        np.nan
    )

print(f"   ✓ Created percentage variables for all cost categories")

# Irrigation system indicator flags
df['is_drip_irrigation'] = (df['irrigation_system_type'] == 'Drip').astype(int)
df['is_traditional_irrigation'] = (df['irrigation_system_type'] == 'Traditional').astype(int)
print(f"   ✓ Created irrigation system indicator variables")

# Date features
if 'survey_date' in df.columns:
    df['survey_year'] = pd.to_datetime(df['survey_date']).dt.year
    df['survey_month'] = pd.to_datetime(df['survey_date']).dt.month
    df['survey_month_name'] = pd.to_datetime(df['survey_date']).dt.month_name()
    print(f"   ✓ Created date-related variables")

3. Creating additional derived variables...
   ✓ Created percentage variables for all cost categories
   ✓ Created irrigation system indicator variables
   ✓ Created date-related variables


## Preview the Dataset

In [5]:
# Display sample of new variables
new_vars = ['total_cost', 'production_kg', 'cost_per_kg', 'production_efficiency']
display(df[['irrigation_system_type', 'crop_type'] + new_vars].head(10))

,irrigation_system_type,crop_type,total_cost,production_kg,cost_per_kg,production_efficiency
0,Traditional,Eggplant,385000.0,120.0,3208.333333,0.311688
1,Drip,Eggplant,194680.0,0.0,NaN,0.000000
2,Traditional,Tomato,955320.0,3200.0,298.537500,3.349663
3,Drip,Tomato,424000.0,800.0,530.000000,1.886792
4,Drip,pepper,539480.0,2000.0,269.740000,3.707274
5,Traditional,pepper,925080.0,860.0,1075.674419,0.929649
6,Drip,pepper,350900.0,NaN,NaN,NaN
7,Drip,pepper,249000.0,0.0,NaN,0.000000
8,Traditional,pepper,288400.0,140.0,2060.000000,0.485437
9,Traditional,pepper,375000.0,200.0,1875.000000,0.533333


## Export Dataset

In [6]:
print("4. Exporting dataset...")

# Export to Excel (primary format for Tableau)
excel_file = 'Irrigation_Analysis_Tableau_Ready.xlsx'
df.to_excel(excel_file, index=False, sheet_name='Irrigation Data')
print(f"   ✓ Exported to Excel: {excel_file}")

# Export to CSV (alternative format)
csv_file = 'Irrigation_Analysis_Tableau_Ready.csv'
df.to_csv(csv_file, index=False, encoding='utf-8')
print(f"   ✓ Exported to CSV: {csv_file}")

4. Exporting dataset...
   ✓ Exported to Excel: Irrigation_Analysis_Tableau_Ready.xlsx
   ✓ Exported to CSV: Irrigation_Analysis_Tableau_Ready.csv


## Summary Report

In [7]:
print(" " + "="*80)
print("SUMMARY")
print("="*80)
print(f"Total Records: {len(df)}")
print(f"Total Variables: {len(df.columns)}")
print(f"Irrigation System Breakdown:")
print(df['irrigation_system_type'].value_counts())
print(f"Crop Type Breakdown:")
print(df['crop_type'].value_counts())
print(f"Missing Values Summary (Top 5):")
missing_summary = df.isnull().sum().sort_values(ascending=False).head()
if missing_summary.sum() > 0:
    print(missing_summary)
else:
    print("   No missing values!")

print(" " + "="*80)
print("✓ TABLEAU-READY DATASET CREATED SUCCESSFULLY!")
print("="*80)
print(f"You can now import '{excel_file}' into Tableau")
print("All original variables plus derived cost-benefit metrics are included.")

SUMMARY
Total Records: 30
Total Variables: 71
Irrigation System Breakdown:
irrigation_system_type
Traditional    15
Drip           15
Name: count, dtype: int64
Crop Type Breakdown:
crop_type
Onion          14
Leafy plant     6
pepper          6
Eggplant        2
Tomato          2
Name: count, dtype: int64
Missing Values Summary (Top 5):
cost_per_kg                               15
pest_control_chemical_materials_cost      11
production_final_harvest_cost_yer_pct      2
production_harvest_frequency_per_month     2
production_quantity_baskets_kg             2
dtype: int64
✓ TABLEAU-READY DATASET CREATED SUCCESSFULLY!
You can now import 'Irrigation_Analysis_Tableau_Ready.xlsx' into Tableau
All original variables plus derived cost-benefit metrics are included.
